#Anggota Kelompok:
Muhammad Farhan (1103220187)

Fasya Burhanis syauqi (1103223054)

#.

# Chapter 2: TensorFlow 2 Basics & Fundamentals

**Book:** TensorFlow in Action  
**Topic:** Core Data Structures, Eager Execution, and Basic Neural Network Operations.

## 1. Chapter Summary
Bab ini membahas fondasi teknis dari TensorFlow 2.x yang menjadi dasar pembuatan model Deep Learning. Berikut adalah poin-poin kuncinya:

* **TensorFlow 2 vs TF 1:** TF 2 menggunakan **Eager Execution** (imperatif) secara default, yang berarti kode dieksekusi baris-per-baris seperti Python biasa, memudahkan debugging. TF 1 menggunakan pendekatan deklaratif (Graph) yang lebih rumit.
* **Building Blocks Utama:**
    * **`tf.Variable`:** Struktur data *mutable* (bisa diubah). Digunakan untuk menyimpan parameter model (bobot/weights dan bias) yang perlu di-update selama training.
    * **`tf.Tensor`:** Struktur data *immutable* (tidak bisa diubah setelah dibuat). Digunakan untuk menyimpan input data dan output operasi.
    * **`tf.Operation`:** Operasi matematika (tambah, kali, matrix multiplication) yang bekerja pada Tensor/Variable.
* **Neural Network Operations:** Bab ini mendemonstrasikan cara kerja layer Neural Network secara manual menggunakan operasi matriks (`tf.matmul`), fungsi aktivasi (`Sigmoid`, `Softmax`), Konvolusi (`tf.nn.convolution`), dan Pooling (`tf.nn.max_pool`).
* **AutoGraph (`@tf.function`):** Fitur dekorator yang mengubah fungsi Python biasa menjadi *Computational Graph* TensorFlow agar performanya lebih cepat (mendekati kecepatan C++).

## 2. Theoretical Explanation (AI Assisted)
*Bagian ini menjelaskan konsep teoritis yang agak sulit di Chapter 2.*

### A. Eager Execution vs Graph Execution
* **Eager Execution (TF 2.x):** Bayangkan Anda sedang mengetik di kalkulator. Anda tekan `2 + 2`, hasilnya langsung keluar `4`. Ini intuitif, mudah di-debug, tapi bisa jadi lambat jika operasinya sangat banyak.
* **Graph Execution (TF 1.x / `@tf.function`):** Bayangkan Anda menulis resep masakan. Anda menulis semua langkahnya dulu, tapi belum ada yang dimasak. Setelah resep (Graph) jadi, baru "mesin" menjalankannya sekaligus. Ini sangat cepat karena komputer bisa mengoptimalkan langkah-langkahnya (misal: menggabungkan dua langkah menjadi satu), tapi sulit di-debug jika ada error di tengah jalan.

### B. Tensor Rank & Reshaping untuk CNN
Dalam operasi gambar (Computer Vision), kita sering melihat tensor berdimensi 4 (Rank 4). Mengapa?
Format standar TensorFlow untuk gambar adalah: **`[Batch_Size, Height, Width, Channels]`**.
* **Batch Size:** Jumlah gambar yang diproses sekaligus (misal: 32 gambar).
* **Height/Width:** Dimensi piksel gambar (misal: 512x512).
* **Channels:** Kedalaman warna (1 untuk Grayscale, 3 untuk RGB).
Itulah mengapa di bagian kode nanti kita perlu melakukan `tf.reshape` menjadi 4 dimensi sebelum melakukan konvolusi.

### C. Mutable vs Immutable
* **`tf.Variable` (Mutable):** Seperti papan tulis kapur. Isinya bisa dihapus dan ditulis ulang (`.assign`). Ini wajib untuk **Weights** karena Neural Network "belajar" dengan cara mengubah-ubah nilai Weights ini terus menerus.
* **`tf.Tensor` (Immutable):** Seperti pahatan batu. Sekali dibuat, nilainya tetap. Jika ingin mengubahnya, kita harus membuat Tensor baru.

In [8]:
import tensorflow as tf
import numpy as np

print(f"TensorFlow Version: {tf.__version__}")


x = np.random.normal(size=[1, 4]).astype('float32')


init = tf.keras.initializers.RandomNormal()
w1 = tf.Variable(init(shape=[4, 3]))
b1 = tf.Variable(init(shape=[1, 3]))


w2 = tf.Variable(init(shape=[3, 2]))
b2 = tf.Variable(init(shape=[1, 2]))


@tf.function
def forward(x, W, b, act):

    return act(tf.matmul(x, W) + b)



h = forward(x, w1, b1, tf.nn.sigmoid)

y = forward(h, w2, b2, tf.nn.softmax)

print("\n--- Output MLP Manual ---")
print("Input shape:", x.shape)
print("Hidden layer output shape:", h.shape)
print("Final Output (Probabilities):\n", y.numpy())

TensorFlow Version: 2.19.0

--- Output MLP Manual ---
Input shape: (1, 4)
Hidden layer output shape: (1, 3)
Final Output (Probabilities):
 [[0.45690602 0.543094  ]]


In [9]:
# --- 3.2 Understanding tf.Variable vs tf.Tensor ---


v = tf.Variable(np.zeros(shape=[2, 2]), dtype='float32')
print("\n--- tf.Variable Example ---")
print("Original Variable:\n", v.numpy())


v[0, 0].assign(99.0)
print("Modified Variable (assign index [0,0]):\n", v.numpy())


t = tf.constant([[1, 2], [3, 4]], dtype='float32')
print("\n--- tf.Tensor Example ---")
print("Original Tensor:\n", t.numpy())


data = tf.constant([[1, 2, 3], [4, 5, 6]], dtype='float32')
sum_all = tf.reduce_sum(data)
mean_col = tf.reduce_mean(data, axis=0)

print("\n--- Math Operations ---")
print("Data:\n", data.numpy())
print("Sum All:", sum_all.numpy())
print("Mean per Column:", mean_col.numpy())


--- tf.Variable Example ---
Original Variable:
 [[0. 0.]
 [0. 0.]]
Modified Variable (assign index [0,0]):
 [[99.  0.]
 [ 0.  0.]]

--- tf.Tensor Example ---
Original Tensor:
 [[1. 2.]
 [3. 4.]]

--- Math Operations ---
Data:
 [[1. 2. 3.]
 [4. 5. 6.]]
Sum All: 21.0
Mean per Column: [2.5 3.5 4.5]


In [10]:

image_dummy = np.random.rand(512, 512).astype('float32')
image_tensor = tf.constant(image_dummy)

image_reshaped = tf.reshape(image_tensor, [1, 512, 512, 1])


laplacian_filter = np.array([
    [-1, -1, -1],
    [-1,  8, -1],
    [-1, -1, -1]
]).astype('float32')

filter_reshaped = tf.reshape(tf.constant(laplacian_filter), [3, 3, 1, 1])


conv_result = tf.nn.convolution(
    input=image_reshaped,
    filters=filter_reshaped,
    padding="VALID"
)


pool_result = tf.nn.max_pool(
    input=conv_result,
    ksize=(1, 2, 2, 1),
    strides=(1, 2, 2, 1),
    padding='VALID'
)

print("\n--- Computer Vision Operations Shapes ---")
print("Original Image Shape:", image_reshaped.shape)
print("After Convolution Shape:", conv_result.shape)
print("After Max Pooling Shape:", pool_result.shape)
print("(Note: Pooling roughly halves the dimension)")


--- Computer Vision Operations Shapes ---
Original Image Shape: (1, 512, 512, 1)
After Convolution Shape: (1, 510, 510, 1)
After Max Pooling Shape: (1, 255, 255, 1)
(Note: Pooling roughly halves the dimension)
